In [1]:
from routines import reset_all_trades, create_new_investor, create_new_stock, print_all
reset_all_trades()
create_new_investor('besta', 100000)
create_new_stock('thepoon', 13.077, 1.04, 3.5, 1000, 0)
create_new_stock('thuya', 13.500, 2.00, 1.1, 1000.0, 0)


In [2]:
from utils import get_net_worth

get_net_worth('besta')

100000.0

In [3]:
from bank import buy_stock
from routines import print_all

print(get_net_worth('besta'))
buy_stock('besta', 'thuya', 5)
print(get_net_worth('besta'))
# print_all()
buy_stock('besta', 'thuya', -5)
# print_all()
print(get_net_worth('besta'))

100000.0
besta has just bought 5 share(s) of thuya for $144.7 !
100001.45
besta has just sold 5 share(s) of thuya for $116.92 !
99972.22
